In [51]:
import os
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<h1> A Markov matrix (or stochastic matrix) </h1>

<p> Gives the probability of reaching a set of states given a set of starting states. Here's a basic example of what such a Markov chain would look if we wanted to model the weather in the evening given the weather in the morning: </p>

<img src="markov.png">

The starting "state" is listed on the left, while the column gives the final outcome. If we wanted to know the probability of Evening Sun given that it there was Morning rain, we would get 0.25 - a 25% chance of that outcome. Note that each row of this matrix is normalized, a statement that if you start in a state, there is a 100% chance that something happens. Nothing tricky here.


We can use this concept to build 

In [141]:
np.random.seed(30)
characters = list('abcdefghijklmnopqrstuvwxyz ')
cipher     = np.random.permutation(characters)

In [142]:
# take jumbled code to english
correct_map = {code:   actual for code, actual in zip(cipher, characters)}
# take english to jumbled code
jumbler     = {actual: code for code, actual in correct_map.items()}
# indices of correct english
alpha_lookup = {letter: i for i, letter in enumerate(characters)}

In [143]:
# TODO: get english transition matrix
# TODO: make a function that calculations likelihood of observations
# TODO: get jumbled matrix transition matrix
# use metropolis hastings to propose char swaps 

In [144]:
def clean_chapter(doc):
    intermediate = re.sub(r'[^a-z]', ' ', doc)  
    return re.sub(r"\s\s+", " ", intermediate)

In [145]:
chapter = np.loadtxt('./moby_dick/chapter_1.txt', dtype=str)
chapter

array(['chapter', '1', 'loomings', ..., 'in', 'the', 'air'], dtype='<U14')

In [146]:
cleaned_chapter = clean_chapter(" ".join(chapter))

In [212]:
def make_cooccurrence_matrix(corpus, smooth_factor=0.01):
    """
    Matrix of counts
    
    first index is starting char, 
    second is following char
    """
    
    counts = np.zeros(shape=(len(characters), len(characters)))
    last = None
    for index, char in enumerate(corpus):
        if index == 0:
            last = alpha_lookup[char]
            continue
            
        current = alpha_lookup[char]
        counts[last, current] += 1
        last = current
    
    return counts + smooth_factor

In [226]:
def make_transition_matrix(corpus, smooth_factor=0.01):
    mx = make_cooccurrence_matrix(corpus, smooth_factor)
    total_ocs = np.sum(mx, axis=1).reshape(-1, 1)
    return mx / total_ocs

In [228]:
tm = make_transition_matrix(cleaned_chapter)

In [233]:
# these should be about the same (up to a smoothing factor)
tm[0, 1], cleaned_chapter.count('ab') / cleaned_chapter.count('a')

(0.028897233350496692, 0.028894472361809045)

In [ ]:
# logprob of observation is np.sum(np.log(make_transition_matrix(english) * make_cooccurrence_matrix(observation)))